In [46]:
import cv2
import numpy as np
import mss
import time
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [47]:
def processImage(img):
    #img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)# 彩色
    # 圖片轉換成灰階色彩
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY); 

    # 使用 CLAHE 增強對比度
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    # 模糊化
    # output = cv2.medianBlur(output, 5)

    # 降低雜訊
    blur = cv2.GaussianBlur(enhanced, (3, 3), 0)

    #自適應二值化
    #binary = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    return binary

In [48]:
def recognizeText(img):
    customConfig = r'--oem 2 --psm 3'  # 文字辨識模式
    text = pytesseract.image_to_string(img, config = customConfig, lang = "chi_tra")
    return text.strip()

In [49]:
def captureScreen():
    cv2.namedWindow("Game Capture")  # 允許視窗縮放
    #cv2.resizeWindow("Game Capture", 800, 600)  # 設定視窗大小

    with mss.mss() as sct:
        monitor = sct.monitors[1]  # 擷取主螢幕（可改成特定區域）
        screenWidth = monitor["width"]
        screenHeight = monitor["height"]
        # 設定擷取區域（右側 1/5）
        captureWidth = screenWidth // 3
        region = {
            "left": 1890,
            "top": 195,
            "width": 300,
            "height": 45
        }
        
        
        while True:
            #start_time = time.time()
            
            # 擷取畫面
            screenshot = sct.grab(region)

            # 轉換成 OpenCV 格式
            img = np.array(screenshot)

            # 影像處理
            processedImg = processImage(img)

            # OCR 文字辨識
            #recognizedText = recognizeText(processedImg)

            # 顯示辨識結果
            #print("辨識結果:", recognizedText)
            
            # 顯示畫面
            cv2.imshow("Game Capture", processedImg)

            # 按 Q 結束
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

            #print(f"FPS: {1 / (time.time() - start_time):.2f}")

    cv2.destroyAllWindows()

In [50]:
captureScreen()